In [4]:
import pandas as pd
import numpy as np
from scipy.special import softmax

from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

## Data loading

In [5]:
path_df = "Data/Table/Pokemon_name_and_type.parquet"

In [6]:
df = pd.read_parquet(path_df)

## Train-test split

In [7]:
targets = ["Type 1", "Type 2"]

X = df.drop(columns=targets)
y = df[targets].apply(lambda x: [x['Type 1']] if pd.isna(x['Type 2']) else [x['Type 1'], x['Type 2']], axis=1)
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modeling

In [8]:
transformer = ColumnTransformer(
    transformers=[
        ("en", TfidfVectorizer(analyzer="char", ngram_range=(1, 3), lowercase=True, max_features=1000), "English"),
        ("de", TfidfVectorizer(analyzer="char", ngram_range=(1, 3), lowercase=True, max_features=1000), "German"),
        ("fr", TfidfVectorizer(analyzer="char", ngram_range=(1, 3), lowercase=True, max_features=1000), "French"),
        ("ka", TfidfVectorizer(analyzer="char", ngram_range=(1, 3), lowercase=True, max_features=1000), "Kanas"),
        ("re", TfidfVectorizer(analyzer="char", ngram_range=(1, 3), lowercase=True, max_features=1000), "Registered")
    ],
    remainder="drop",
    sparse_threshold=0,
)

model = MultiOutputClassifier(LogisticRegression(max_iter=1000, class_weight="balanced"))

pipe = make_pipeline(transformer, model)
pipe.fit(X_train, y_train);

In [9]:
pred_probas = pipe.predict_proba(X_test)
pred_probas = np.array(pred_probas)[:,:,1].T
#pred_probas = np.mean(pred_probas, axis=1)

In [21]:
row_sums = pred_probas.sum(axis=1).reshape(-1, 1)  # Reshape for broadcasting

# Divide each row by its sum
(pred_probas / row_sums).sum(1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

In [11]:
's' in {'s', None, 3}

True

In [12]:
threshold = .07
preds = np.zeros_like(pred_probas)

for i,pred_proba in enumerate(pred_probas) :
    t = np.argsort(pred_proba)
    preds[i, t[-1]] = 1
    second_best = t[-2]
    if pred_proba[second_best] > threshold:
        preds[i, second_best] = 1

In [13]:
y_pred = pipe.predict(X_test)

print(classification_report(y_test, y_pred, target_names=mlb.classes_, zero_division=0))

              precision    recall  f1-score   support

         Bug       0.75      0.17      0.27        18
        Dark       0.25      0.07      0.11        15
      Dragon       0.70      0.54      0.61        13
    Electric       0.90      0.56      0.69        16
       Fairy       0.75      0.27      0.40        11
    Fighting       0.25      0.11      0.15        18
        Fire       1.00      0.33      0.50        12
      Flying       0.67      0.38      0.48        16
       Ghost       1.00      0.07      0.13        14
      Ground       0.36      0.25      0.30        16
         Ice       1.00      0.14      0.25         7
      Normal       0.57      0.35      0.43        23
       Plant       0.47      0.27      0.34        26
      Poison       0.83      0.42      0.56        24
     Psychic       0.50      0.27      0.35        22
        Rock       0.60      0.43      0.50        14
       Steel       0.67      0.31      0.42        13
       Water       0.67    

In [14]:
accuracy_score(y_test, y_pred)

0.21951219512195122